# Data analysis project

Written by Julien Samuel, Mayuxiang Chen and Jolan Kerrien

In [10]:
import numpy as np
from scipy import stats
from math import sqrt
from scipy.stats import chi2_contingency
import pandas as pd
#Ploting librairies:
import matplotlib.pyplot as plt
import seaborn as sns
import pingouin as pg

## 1) Graphical Representation of all the variables with respect to the target + Computing the correlation

Functions that will help us through our analysis

In [11]:
# Importation of the data 
df = pd.read_csv('energy_data.csv', index_col = 0).reset_index(drop = True)
df.head()

,X2,X3,X4,X5,X6,X7,X8,Y1,Y2
0,514.5,294.0,110.25,7.0,North,0.0,type1,15.55,21.33
1,514.5,294.0,110.25,7.0,South,0.0,type1,15.55,21.33
2,514.5,294.0,110.25,7.0,Est,0.0,type1,15.55,21.33
3,514.5,294.0,110.25,7.0,West,0.0,type1,15.55,21.33
4,563.5,318.5,122.50,7.0,North,0.0,type1,20.84,28.28


In [12]:
def identify_missing(data): 
    ''' Identify the features missing values NaN
    Return list of features with the number of missing values 
    input = dataframe and print dataframe '''
    missing = data.isnull().sum()
    missing_result = pd.DataFrame(missing).reset_index().rename(columns = {'index' : 'features', 0 :'missing_nbr'})
    missing_result['missing_fraction']=(missing_result.missing_nbr/data.shape[0])*100
    print(missing_result)
    
def information_about_data(data): 
	'''numeric features : get the min and the max
       categorical features get the nbr of classes
       input = a dataframe and prints statistics of columns of this dataframe'''
	print(" ")
	print("Numerical features")
	for col in data.columns : 
		if data[col].dtype != 'object' :
			print('The feature "%s" have a minimum value of %s and a maximum value of %s' % (col,data[col].min(), data[col].max()))

	print(" ")
	print("Categorical features")
	for col in data.columns : 
		if data[col].dtype == 'object' :
			print('The feature "%s" have %s unique values' % (col,len(data[col].unique())))

In [13]:
identify_missing(df)

  features  missing_nbr  missing_fraction
0       X2            0               0.0
1       X3            0               0.0
2       X4            0               0.0
3       X5            0               0.0
4       X6            0               0.0
5       X7            0               0.0
6       X8            0               0.0
7       Y1            0               0.0
8       Y2            0               0.0


There is no missing numbers in our data, so no need to think about a way of replaced them

In [14]:
information_about_data(df)

 
Numerical features
The feature "X2" have a minimum value of 514.5 and a maximum value of 808.5
The feature "X3" have a minimum value of 245.0 and a maximum value of 416.5
The feature "X4" have a minimum value of 110.25 and a maximum value of 220.5
The feature "X5" have a minimum value of 3.5 and a maximum value of 7.0
The feature "X7" have a minimum value of 0.0 and a maximum value of 0.4
The feature "Y1" have a minimum value of 6.01 and a maximum value of 43.1
The feature "Y2" have a minimum value of 10.9 and a maximum value of 48.03
 
Categorical features
The feature "X6" have 4 unique values
The feature "X8" have 6 unique values


As introduce in the data